# Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns 
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
# Importing the libraries for evaluation
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.metrics import (precision_score, recall_score,f1_score)
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_recall_curve
from imblearn.metrics import geometric_mean_score

# Loading the dataset

In [2]:
def load_dataset():
    X_train_load = np.loadtxt('data\X_train_reshaped_multi.csv', delimiter=',')
    X_train_scaled = np.reshape(X_train_load, (X_train_load.shape[0], X_train_load.shape[1], 1))   
    X_test_load = np.loadtxt('data\X_test_reshaped_multi.csv', delimiter=',')
    X_test_scaled = np.reshape(X_test_load, (X_test_load.shape[0], X_test_load.shape[1], 1))  
    y_train_scaled = np.loadtxt('data\y_train_reshaped_multi.csv', delimiter=',')
    y_test_scaled = np.loadtxt('data\y_test_reshaped_multi.csv', delimiter=',')
    X_val_load = np.loadtxt('data\X_val_reshaped_multi.csv', delimiter=',')
    X_val_scaled = np.reshape(X_val_load, (X_val_load.shape[0], X_val_load.shape[1], 1))
    y_val_scaled = np.loadtxt('data\y_val_reshaped_multi.csv', delimiter=',')
    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, X_val_scaled, y_val_scaled

# Creating the LSTM model for multi-class classification

In [3]:
def create_model(X_train_scaled):
    model = Sequential() 
    # Adding the first LSTM layer and Dropout regularization
    model.add(LSTM(units= 76, return_sequences= True, input_shape=  ( X_train_scaled.shape[1], 1)))
    model.add(Dropout(0.2))
    # Adding the second LSTM layer and Dropout regularization
    model.add(LSTM(units= 76, return_sequences= True))
    model.add(Dropout(0.2))   
    # Adding the third LSTM layer and Dropout regularization
    model.add(LSTM(units= 76, return_sequences= True))
    model.add(Dropout(0.2))   
    # Adding the fourth LSTM layer and Dropout regularization
    model.add(LSTM(units= 76))
    model.add(Dropout(0.2))    
    # Adding the output layer
    model.add(Dense(units= 15))
    model.add(Activation('softmax'))   
    opt = Adam(lr=0.00002)    
    # Compiling the LSTM
    model.compile(optimizer= opt, loss= 'categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

# Training the model

In [4]:
def train_model(model, X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled):
    earlystopping = callbacks.EarlyStopping(monitor ="val_loss",
    										mode ="min", patience = 5,
    										restore_best_weights = True)  
    hist = model.fit(X_train_scaled, y_train_scaled, batch_size = 1024, epochs = 40, validation_data =(X_val_scaled, y_val_scaled), callbacks = earlystopping)    
    fin_epoch = earlystopping.stopped_epoch
    return(hist, fin_epoch)

# Evaluating the model

In [5]:
def evaluate_model(X_test_scaled, y_test_scaled, model, hist):
    # Predicting values
    y_pred = model.predict_classes(X_test_scaled)
    n_values = np.max(y_pred) + 1
    y_prednew = np.eye(n_values)[y_pred]
    y_prednew = np.reshape(y_prednew, (y_prednew.shape[0], -1))
    y_testnew = np.where(y_test_scaled==1)[1]
    y_prednew2 = model.predict(X_test_scaled)
    # Calculating the performance metrics
    training_loss = hist.history['loss']
    training_acc = hist.history['accuracy']
    loss, accuracy = model.evaluate(X_test_scaled, y_test_scaled)
    balanced_accuracy = balanced_accuracy_score(y_testnew, y_pred)
    gmean_score = geometric_mean_score(y_testnew, y_pred)
    recall = recall_score(y_test_scaled, y_prednew , average="weighted")
    precision = precision_score(y_test_scaled, y_prednew , average="weighted")
    f1 = f1_score(y_test_scaled, y_prednew, average="weighted") 
    print("Training Loss:", training_loss)
    print("Training Accuracy:", training_acc)
    print("Overall Accuracy:", accuracy)
    print("Overall Loss:", loss)
    print("Balanced Accuracy:", balanced_accuracy)
    print("Geometric Mean:", gmean_score)
    print("Recall:", recall)
    print("Precision:", precision)
    print("F1 Score:", f1)
    # Multiclass Confusion Matrix
    multi_cm = multilabel_confusion_matrix(y_test_scaled, y_prednew)
    return(y_pred, y_prednew, y_prednew2, multi_cm, training_loss, training_acc)

# Plotting the results

In [6]:
# Plot Training Accuracy & Loss vs. Epochs
def plot_acc_loss(fin_epoch, training_loss, training_acc):
    if fin_epoch > 0:     
        epoch = fin_epoch
    else:
        epoch = 40
    xc = range(epoch)
    plt.figure(1,figsize=(15,epoch)) 
    plt.plot(xc,training_loss)
    plt.xlabel('No. of Epochs') 
    plt.ylabel('loss') 
    plt.title('Training Loss') 
    plt.grid(True) 
    plt.legend(['Train'])    
    plt.figure(2,figsize=(15,epoch)) 
    plt.plot(xc,training_acc) 
    plt.xlabel('No. of Epochs') 
    plt.ylabel('Accuracy') 
    plt.title('Training Accuracy') 
    plt.grid(True) 
    plt.legend(['Train'],loc=4)

In [7]:
# Plot the confusion matrix wrt one-vs-rest
def calc_cm(multi_cm, axes, label, class_names, fontsize=25):
    df_cm = pd.DataFrame(
        multi_cm, index=class_names, columns=class_names)
    try:
        sns.set(font_scale=2.2)
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes, cmap="Blues")
    except ValueError:
        raise ValueError("CM values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("CM for the class - " + label)

In [8]:
# Plot ROC Curve
def plot_roc_auc(y_test_scaled, y_prednew2, class_labels):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes = len(class_labels)
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_scaled[:, i], y_prednew2[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_scaled.ravel(), y_prednew2.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])  
    # Compute macro-average value for ROC curve and ROC area 
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))    
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes  
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    # Plot all ROC curves
    fig = plt.figure(figsize=(8,6))
    plt.tick_params(axis='both', which='major', labelsize=13)
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = plt.cm.jet(np.linspace(0, 1, 15))
    for i,color in zip(range(n_classes), colors):
        plt.plot(fpr[i], 
                 tpr[i], 
                 color=color,
                 lw=1.5,
                 label="{}, AUC={:.3f}".format(class_labels[i], roc_auc[i]))
    plt.plot([0,1], [0,1], color='orange', linestyle='--') 
    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("False Positive Rate", fontsize=15)
    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)
    plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [9]:
# Plot PR curve
def plot_pr_auc(y_test_scaled, y_prednew, class_labels):
    precision = dict()
    recall = dict()
    pr_auc = dict()
    n_classes = len(class_labels)
    colors = plt.cm.jet(np.linspace(0, 1, 15))
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test_scaled[:, i], y_prednew[:, i])
        pr_auc[i] = auc(recall[i], precision[i])
    fig = plt.figure(figsize=(8,6))
    plt.tick_params(axis='both', which='major', labelsize=13)
    for i,color in zip(range(n_classes), colors):
        plt.plot(recall[i], 
                 precision[i], 
                 color=color,
                 lw=1.5,
                 label="{}, AUC={:.3f}".format(class_labels[i], pr_auc[i]))
    plt.plot([0,1], [0.5,0.5], color='orange', linestyle='--')
    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("Recall Rate", fontsize=15)
    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("Precision Rate", fontsize=15)
    plt.title('Precision Recall Curve', fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
def main():
    class_labels = ["Benign",
                    "Bot",
                    "Brute Force -Web",
                    "Brute Force -XSS",
                    "DDOS attack-HOIC",
                    "DDOS attack-LOIC-UDP",
                    "DDoS attacks-LOIC-HTTP",
                    "DoS attacks-GoldenEye",
                    "DoS attacks-Hulk",
                    "DoS attacks-SlowHTTPTest",
                    "DoS attacks-Slowloris",
                    "FTP-BruteForce",
                    "Infiltration",
                    "SQL Injection",
                    "SSH-Bruteforce"]
    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, X_val_scaled, y_val_scaled = load_dataset()    
    model = create_model(X_train_scaled)
    hist = train_model(model, X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled)
    y_pred, y_prednew, y_prednew2, multi_cm, class_report = evaluate_model(X_test_scaled, y_test_scaled, model, hist)
    # Plot Classification Report
    sns.set(font_scale=0.8)
    sns.heatmap(pd.DataFrame(class_report).iloc[:-1, :].T, annot=True)
    # Plot Confusion Matrix
    fig, ax = plt.subplots(2, 2, figsize=(20, 20))
    for axes, cfs_matrix, label in zip(ax.flatten(), multi_cm, class_labels):
        print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    fig.tight_layout()
    plt.show()
    # Plot ROC Curve
    plot_roc_auc(y_test_scaled, y_prednew2, class_labels)
    # Plot PR Curve
    plot_pr_auc(y_test_scaled, y_prednew, class_labels)